In [1]:
# Load all dictionaries
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import scikit_posthocs as sp
from scipy.stats import wilcoxon
from heatmaps_functions import *
from linares_plot import *

sns.set_context("poster", font_scale=1.1)
sns.set_style("ticks")

In [2]:
paths_target=[ 'C:\\Users\\David\\Desktop\\KI_Desktop\\data_reconstructions\\IEM\\IEM_target_far_delay_NT.xlsx',
              'C:\\Users\\David\\Desktop\\KI_Desktop\\data_reconstructions\\IEM\\IEM_target_close_delay_NT.xlsx']


paths_target_shuff=[ 'C:\\Users\\David\\Desktop\\KI_Desktop\\data_reconstructions\\IEM\\shuff_IEM_target_far_delay_NT.xlsx',
              'C:\\Users\\David\\Desktop\\KI_Desktop\\data_reconstructions\\IEM\\shuff_IEM_target_close_delay_NT.xlsx']

In [3]:
df_target=[]
df_target_shuff=[]

for PathI in paths_target:
    path_save_signal =PathI
    #
    dec_thing = path_save_signal.split('\\')[-1].split('.')[0].split('_')[1]
    distance = path_save_signal.split('\\')[-1].split('.')[0].split('_')[2]
    training_region = path_save_signal.split('\\')[-1].split('.')[0].split('_')[3]
    #
    df_x = pd.read_excel(PathI)
    df_x['item'] = dec_thing
    df_x['trained'] =training_region
    df_x['distance'] = distance
    #
    df_target.append(df_x)
    
#
df_target=pd.concat(df_target, axis=0)

####################

for PathI in paths_target_shuff:
    path_save_signal =PathI
    #
    dec_thing = path_save_signal.split('\\')[-1].split('.')[0].split('_')[2]
    distance = path_save_signal.split('\\')[-1].split('.')[0].split('_')[3]
    training_region = path_save_signal.split('\\')[-1].split('.')[0].split('_')[4]
    #
    df_x = pd.read_excel(PathI)
    df_x['item'] = dec_thing
    df_x['trained'] =training_region
    df_x['distance'] = distance
    #
    df_target_shuff.append(df_x)
    
#
df_target_shuff=pd.concat(df_target_shuff, axis=0)

In [4]:
df = df_target
df['label']= df['item'] + ' ' + df['distance'] 
df['label'] = df['label'].replace(['target close', 'target far', 'dist close', 'dist far'], ['tc', 'tf', 'dc', 'df'])
df.head()

,times,decoding,region,subject,condition,label,item,trained,distance
0,0.000,-3.620059,visual,l001,1_0.2,tf,target,delay,far
1,2.335,-2.554755,visual,l001,1_0.2,tf,target,delay,far
2,4.670,1.465164,visual,l001,1_0.2,tf,target,delay,far
3,7.005,4.434589,visual,l001,1_0.2,tf,target,delay,far
4,9.340,2.861907,visual,l001,1_0.2,tf,target,delay,far


In [5]:
df_shuff = df_target_shuff
df_shuff['label']= df_shuff['item'] + ' ' + df_shuff['distance'] 
df_shuff['label'] = df_shuff['label'].replace(['target close', 'target far', 'dist close', 'dist far'], ['tc', 'tf', 'dc', 'df'])
df_shuff.head()

,times,decoding,region,subject,condition,label,item,trained,distance
0,0.000,-0.786574,visual,d001,1_0.2,tf,target,delay,far
1,2.335,-0.897154,visual,d001,1_0.2,tf,target,delay,far
2,4.670,-0.186175,visual,d001,1_0.2,tf,target,delay,far
3,7.005,0.435306,visual,d001,1_0.2,tf,target,delay,far
4,9.340,1.188504,visual,d001,1_0.2,tf,target,delay,far


In [6]:
df_delay=[]
for cond in ['1_0.2', '2_0.2']:
    for reg in ['visual', 'ips', 'pfc']:
        for subj in ['d001', 's001', 'l001', 'n001', 'b001', 'r001']:
            for dist in ['close','far']: 
                for item in ['target']:
                    ####
                    if cond == '1_0.2':
                        dfn = df.loc[(df['region']==reg) & 
                                 ( (df['times']>= 9.34) & (df['times']<= 18.688) ) & 
                                    (df['condition']==cond) & (df['subject']==subj) &  
                                 ( df['distance']==dist) & (df['trained']=='delay') & 
                                 ( df['item']==item)]
                        #
                        dfn_shuff = df_shuff.loc[(df_shuff['region']==reg) & 
                                 ( (df_shuff['times']>= 9.34) & (df_shuff['times']<= 18.688) ) & 
                                 ( df_shuff['condition']==cond) & (df_shuff['subject']==subj) &  
                                 ( df_shuff['distance']==dist) & (df_shuff['trained']=='delay') & 
                                 ( df_shuff['item']==item)]

                        ###
                        decoding = dfn.decoding.mean() #option1
                        #decoding_sh = dfn.decoding.mean() - dfn_shuff.decoding.mean() #option2
                        decoding_sh = (dfn.decoding.mean() - dfn_shuff.decoding.mean()) / dfn_shuff.decoding.std()  #option2
                        ###
                        df_delay.append( [ 'delay', decoding, decoding_sh, reg, subj, cond, dfn.label.iloc[0],
                                      item, 'delay', dist] )
                        #                    
                    elif cond == '2_0.2':
                        dfn = df.loc[(df['region']==reg) & 
                                 ( (df['times']>= 9.34) & (df['times']<= 18.688) ) & 
                                 ( df['condition']==cond) & (df['subject']==subj) &  
                                 ( df['distance']==dist) & (df['trained']=='delay') & 
                                 ( df['item']==item)]
                        #
                        dfn_shuff = df_shuff.loc[(df_shuff['region']==reg) & 
                                 ( (df_shuff['times']>= 9.34) & (df_shuff['times']<= 18.688) ) & 
                                 ( df_shuff['condition']==cond) & (df_shuff['subject']==subj) &  
                                 ( df_shuff['distance']==dist) & (df_shuff['trained']=='delay') & 
                                 ( df_shuff['item']==item) ]

                        ###
                        decoding = dfn.decoding.mean() #option1
                        #decoding_sh = dfn.decoding.mean() - dfn_shuff.decoding.mean() #option2
                        decoding_sh = (dfn.decoding.mean() - dfn_shuff.decoding.mean()) / dfn_shuff.decoding.std() 
                        ###
                        df_delay.append( [ 'delay', decoding, decoding_sh, reg, subj, cond, dfn.label.iloc[0], 
                                          item, 'delay', dist] )
                        ##
                    ##
                ##
            ##
        ##
    ##
##
##
##

df_delay = pd.DataFrame(df_delay)
df_delay.columns=['times', 'decoding', 'decoding_sh','region', 'subject', 'condition', 'label', 'item', 'trained', 'distance']
